In [1]:
from imagededup.hashing import Hashing
from imagededup.retrieve.retrieval import HashEval
from imagededup.evaluation import EvalPerformance
from pickle import load as pickle_loader
from datetime import datetime
from pathlib import Path

QUERY_PATH = Path('/Users/tanuj.jain/Documents/dedup-data/Transformed_dataset/Query')
TEST_PATH = Path('/Users/tanuj.jain/Documents/dedup-data/Transformed_dataset/Retrieval')
GOLD_PATH = Path('/Users/tanuj.jain/Documents/dedup-data/Transformed_dataset/ground_truth_transformed.pkl')

In [2]:
hasher = Hashing(method='phash')
start = datetime.utcnow()
hash_dict_query = hasher.hash_dir(QUERY_PATH)
print(f'Stage 1: FINGEPRINTING query completed in {(datetime.utcnow() - start).total_seconds()} seconds')

start = datetime.utcnow()
hash_dict_test = hasher.hash_dir(TEST_PATH)
print(f'Stage 1: FINGEPRINTING test completed in {(datetime.utcnow() - start).total_seconds()} seconds')

start = datetime.utcnow()
rs = HashEval(test=hash_dict_test, queries=hash_dict_query, hammer=hasher.hamming_distance, cutoff=13)
returned_dict = rs.retrieve_result_list()
print(f'Stage 2: SEARCH & RETRIEVAL completed in {(datetime.utcnow() - start).total_seconds()} seconds')

2019-07-16 18:57:02,917 imagededup.hashing: INFO Start: Calculating hashes using phash!
2019-07-16 18:57:36,712 imagededup.hashing: INFO End: Calculating hashes using phash!


Stage 1: FINGEPRINTING query completed in 33.854523 seconds


2019-07-16 18:57:36,992 imagededup.hashing: INFO Start: Calculating hashes using phash!
2019-07-16 18:59:49,801 imagededup.hashing: INFO End: Calculating hashes using phash!
2019-07-16 18:59:49,805 imagededup.retrieve.retrieval: INFO Start: Retrieving duplicates using BKTree algorithm


Stage 1: FINGEPRINTING test completed in 133.09078 seconds


2019-07-16 19:07:07,604 imagededup.retrieve.retrieval: INFO End: Retrieving duplicates using BKTree algorithm


Stage 2: SEARCH & RETRIEVAL completed in 437.804258 seconds


In [3]:
with open(GOLD_PATH, 'rb') as buf:
    correct_dict = pickle_loader(buf)

In [4]:
evaluator = EvalPerformance(correct_dict, returned_dict)
for metric, reading in evaluator.get_all_metrics().items():
    print(f'{metric}\t{reading}')

MAP	0.2660700316365833
NDCG	0.8179152264209751
Jaccard	0.21795808562656768
